# Introduction

THis notebook contains he final notebook submission to the Kaggle competion predcit Calories challenge. I will start by first havoing code to the best possible solution based on the public leaderboard) and then explore other options. EDA and other stuff have been removed from this notebook for readability.

In [2]:
# imports

import pandas as pd
import numpy as np

In [3]:
train = pd.read_csv('PredictCalorie/train.csv')
test = pd.read_csv('PredictCalorie/test.csv')
sample = pd.read_csv('PredictCalorie/sample_submission.csv')

# Use of CatBoost

In [ ]:
# Define columns to exclude from features
exclude_cols = ['id', 'Calories']
target_col = 'Calories'

# Split features and target from train
X_train = train.drop(columns=exclude_cols)
y_train = train[target_col]

# Prepare test set (same feature columns)
X_test = test.drop(columns=['id'])

# Identify categorical features (assumes object types are categorical)
categorical_features = ['Sex']

# Optional: split validation set from train (for early stopping)
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Define Pools
train_pool = Pool(X_tr, y_tr, cat_features=categorical_features)
val_pool = Pool(X_val, y_val, cat_features=categorical_features)
test_pool = Pool(X_test, cat_features=categorical_features)

# Train the model
model = CatBoostRegressor(
    iterations=2000,
    learning_rate=0.03,
    depth=10,
    random_strength=5,
    l2_leaf_reg=7,
    bootstrap_type='Bernoulli',
    grow_policy='SymmetricTree',
    eval_metric='RMSE',
    random_seed=42,
    early_stopping_rounds=50,
    verbose=100
)

model.fit(train_pool, eval_set=val_pool)

# Predict on test set
test_preds = model.predict(test_pool)

# Add predictions to test dataframe
test['Calories'] = test_preds

# Optional: preview results
print(test[['id','Calories']].head())
test[['id','Calories']].to_csv('catboost.csv',index=False)

In [ ]:
from sklearn.model_selection import StratifiedKFold
import pandas as pd
import numpy as np

# Bin 'Duration' into quantile-based categories (e.g., 5 groups)
X_train = train.drop(columns=exclude_cols)
y_train = train[target_col]

# Add Duration binning for stratification
duration_bins = pd.qcut(train['Duration'], q=10, labels=False)  # or use pd.cut for fixed bins

# Initialize StratifiedKFold using duration bins
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

test_preds = np.zeros(len(test))


# Proceed with the same logic — just replace KFold with StratifiedKFold
for fold, (idx_train, idx_valid) in enumerate(cv.split(X_train, duration_bins)):
    print(f'\nFold {fold + 1}')
    
    X_tr, X_val = X_train.iloc[idx_train], X_train.iloc[idx_valid]
    y_tr, y_val = np.log1p(y_train.iloc[idx_train]), np.log1p(y_train.iloc[idx_valid])
    
    # CatBoost Pools
    train_pool = Pool(X_tr, y_tr, cat_features=categorical_features)
    val_pool = Pool(X_val, y_val, cat_features=categorical_features)
    test_pool = Pool(X_test, cat_features=categorical_features)
    
    # Train and predict as before...
    model = CatBoostRegressor(
        iterations=2000,
        learning_rate=0.03,
        depth=10,
        random_strength=5,
        l2_leaf_reg=7,
        bootstrap_type='Bernoulli',
        grow_policy='SymmetricTree',
        eval_metric='RMSE',
        random_seed=42,
        early_stopping_rounds=50,
        verbose=100
    )
    
    model.fit(train_pool, eval_set=val_pool)
    test_preds += np.expm1(model.predict(test_pool))

test_preds /= cv.get_n_splits()
test['Calories'] = test_preds
test[['id','Calories']].to_csv('catboost_5_fold_str.csv',index=False)

# Add more features

In [ ]:
train = pd.read_csv('PredictCalorie/train.csv')
test = pd.read_csv('PredictCalorie/test.csv')
sample = pd.read_csv('PredictCalorie/sample_submission.csv')
train.head()

In [ ]:
train['bmi']=train['Weight']/(train['Height']*train['Height'])*100*100
test['bmi']=test['Weight']/(test['Height']*test['Height'])*100*100

train['dur_body_heart']=train['Duration']*train['Heart_Rate']*train['Body_Temp']
test['dur_body_heart']=test['Duration']*test['Heart_Rate']*test['Body_Temp']



In [ ]:
def calculate_bmr(row):
    if row['Sex'].lower() == 'male':
        return 10 * row['Weight'] + 6.25 * row['Height'] - 5 * row['Age'] + 5
    elif row['Sex'].lower() == 'female':
        return 10 * row['Weight'] + 6.25 * row['Height'] - 5 * row['Age'] - 161
    else:
        return np.nan  # handle unknown or missing Sex

train['BMR'] = train.apply(calculate_bmr, axis=1)
test['BMR'] = test.apply(calculate_bmr, axis=1)

In [ ]:
train.head()

# XGB Regressor

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import TargetEncoder
import xgboost as xgb
import numpy as np
import pandas as pd

categorical_cols = ['Sex']

# Discretize 'duration' into 5 bins for stratification
duration_bins = pd.qcut(X['Duration'], q=5, labels=False, duplicates='drop')

# Use StratifiedKFold for stratified splitting based on duration bins
cv = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
y_pred = np.zeros(len(test))

# Log-transform the target
y_log = np.log1p(y)  # log(1 + y) for RMSLE-friendly training

for idx_train, idx_valid in cv.split(X, duration_bins):
    X_train, y_train = X.iloc[idx_train].copy(), y_log.iloc[idx_train]
    X_valid, y_valid = X.iloc[idx_valid].copy(), y_log.iloc[idx_valid]
    X_test = test[X.columns].copy()

    # Target Encoding for categorical column
    encoder = TargetEncoder(random_state=42)
    X_train['Sex'] = encoder.fit_transform(X_train[['Sex']], y_train)[:, 0]
    X_valid['Sex'] = encoder.transform(X_valid[['Sex']])[:, 0]
    X_test['Sex'] = encoder.transform(X_test[['Sex']])[:, 0]

    model = xgb.XGBRegressor(
        n_estimators=2000,
        max_depth=10,
        learning_rate=0.03,
        subsample=0.7,
        colsample_bytree=0.7,
        objective='reg:squarederror',  # L2 loss
        eval_metric='rmse',
        early_stopping_rounds=100,
        random_state=42,
        verbosity=1,
    )

    model.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        verbose=100
    )

    y_pred += np.expm1(model.predict(X_test))

# Average predictions over folds
y_pred /= cv.get_n_splits()

# Create submission file
pred_xgb = y_pred
sample_submission = pd.read_csv('sample_submission.csv')
submission_xgb = pd.DataFrame({'id': sample_submission.id, 'Calories': pred_xgb})
submission_xgb.to_csv('submission_xgb.csv', index=False)

# ENsemble the two best cases (In log space)

In [ ]:
data1=pd.read_csv('submission_xgb (1).csv')
data2=pd.read_csv('catboost_5_fold_str (1).csv')

data = data1.merge(data2, on='id', suffixes=('_xgb', '_cat'))

In [ ]:
data.head()

In [ ]:
import pandas as pd
import numpy as np



# 1. Simple average of the two predictions
data['Calories_avg'] = (data['Calories_xgb'] + data['Calories_cat']) / 2

# 2. Log-scale average (RMSLE-style averaging)
data['Calories_log_avg'] = np.expm1(
    (np.log1p(data['Calories_xgb']) + np.log1p(data['Calories_cat'])) / 2
)

print(data[['id', 'Calories_xgb', 'Calories_cat', 'Calories_avg', 'Calories_log_avg']].head())

In [ ]:
# Save simple average with 'Calories' as the second column
data[['id', 'Calories_avg']].rename(columns={'Calories_avg': 'Calories'}).to_csv('submission_reg_avg.csv', index=False)

# Save log-scale average with 'Calories' as the second column
data[['id', 'Calories_log_avg']].rename(columns={'Calories_log_avg': 'Calories'}).to_csv('submission_log_avg.csv', index=False)

# Start fresh with new features

In [11]:
train = pd.read_csv('PredictCalorie/train.csv')
test = pd.read_csv('PredictCalorie/test.csv')
sample = pd.read_csv('PredictCalorie/sample_submission.csv')



In [12]:
train['bmi']=train['Weight']/(train['Height']*train['Height'])*100*100
test['bmi']=test['Weight']/(test['Height']*test['Height'])*100*100

train['dur_body_heart']=train['Duration']*train['Heart_Rate']*train['Body_Temp']
test['dur_body_heart']=test['Duration']*test['Heart_Rate']*test['Body_Temp']

def calculate_bmr(row):
    if row['Sex'].lower() == 'male':
        return 10 * row['Weight'] + 6.25 * row['Height'] - 5 * row['Age'] + 5
    elif row['Sex'].lower() == 'female':
        return 10 * row['Weight'] + 6.25 * row['Height'] - 5 * row['Age'] - 161
    else:
        return np.nan  # handle unknown or missing Sex

train['BMR'] = train.apply(calculate_bmr, axis=1)
test['BMR'] = test.apply(calculate_bmr, axis=1)

# Squares of features
train['dur_2']=train['Duration']*train['Duration']
test['dur_2']=test['Duration']*test['Duration']

train['hr_2']=train['Heart_Rate']*train['Heart_Rate']
test['hr_2']=test['Heart_Rate']*test['Heart_Rate']

train['height_2']=train['Height']*train['Height']
test['height_2']=test['Height']*test['Height']

train['weight_2']=train['Weight']*train['Weight']
test['weight_2']=test['Weight']*test['Weight']

# Square root of Duration
train['dur_sqrt'] = train['Duration'] ** 0.5
test['dur_sqrt'] = test['Duration'] ** 0.5

train['hr_sqrt'] = train['Heart_Rate'] ** 0.5
test['hr_sqrt'] = test['Heart_Rate'] ** 0.5

train['weight_sqrt'] = train['Weight'] ** 0.5
test['weight_sqrt'] = test['Weight'] ** 0.5

train['height_sqrt'] = train['Height'] ** 0.5
test['height_sqrt'] = test['Height'] ** 0.5

train['Intensity'] = train['Heart_Rate'] / train['Duration']
test['Intensity'] = test['Heart_Rate'] / test['Duration']

# Create binary indicators
train['is_male'] = (train['Sex'].str.lower() == 'male').astype(int)
train['is_female'] = (train['Sex'].str.lower() == 'female').astype(int)

test['is_male'] = (test['Sex'].str.lower() == 'male').astype(int)
test['is_female'] = (test['Sex'].str.lower() == 'female').astype(int)

# Numerical columns to interact with sex
num_cols = ['Age', 'Duration', 'Heart_Rate', 'Weight', 'Height', 'Body_Temp','bmi','Intensity','dur_body_heart']

# Create interaction features
for col in num_cols:
    train[f'{col}_Male'] = train[col] * train['is_male']
    train[f'{col}_Female'] = train[col] * train['is_female']

    test[f'{col}_Male'] = test[col] * test['is_male']
    test[f'{col}_Female'] = test[col] * test['is_female']


for col in ['Weight', 'Height', 'Duration', 'Heart_Rate', 'Body_Temp']:
    train[f'{col}_log'] = np.log1p(train[col])
    test[f'{col}_log'] = np.log1p(test[col])

train['Weight_Height_ratio'] = train['Weight'] / train['Height']
test['Weight_Height_ratio'] = test['Weight'] / test['Height']

train['HeartRate_BodyTemp_ratio'] = train['Heart_Rate'] / train['Body_Temp']
test['HeartRate_BodyTemp_ratio'] = test['Heart_Rate'] / test['Body_Temp']


train['effort_per_kg'] = train['Duration'] * train['Heart_Rate'] / train['Weight']
test['effort_per_kg'] = test['Duration'] * test['Heart_Rate'] / test['Weight']


train['Age_HeartRate'] = train['Age'] * train['Heart_Rate']
test['Age_HeartRate'] = test['Age'] * test['Heart_Rate']

train['Temp_Height'] = train['Body_Temp'] * train['Height']
test['Temp_Height'] = test['Body_Temp'] * test['Height']

from itertools import combinations

poly_cols = ['Age', 'Duration', 'Heart_Rate', 'Weight', 'Height']
for col1, col2 in combinations(poly_cols, 2):
    train[f'{col1}_x_{col2}'] = train[col1] * train[col2]
    test[f'{col1}_x_{col2}'] = test[col1] * test[col2]
'''
train['Duration_2'] = train['Duration']
test['Duration_2'] = test['Duration']

train['Age_2'] = train['Age']
test['Age_2'] = test['Age']
'''

"\ntrain['Duration_2'] = train['Duration']\ntest['Duration_2'] = test['Duration']\n\ntrain['Age_2'] = train['Age']\ntest['Age_2'] = test['Age']\n"

In [ ]:
train.columns

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import TargetEncoder
import xgboost as xgb
import numpy as np
import pandas as pd


drop_cols = ['id', 'Calories']
X = train.drop(columns=drop_cols)
y = train['Calories']
cat_features = ['Sex']

categorical_cols = ['Sex']

# Discretize 'duration' into 5 bins for stratification
duration_bins = pd.qcut(X['Duration'], q=10, labels=False, duplicates='drop')

# Use StratifiedKFold for stratified splitting based on duration bins
cv = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)
y_pred = np.zeros(len(test))

# Log-transform the target
y_log = np.log1p(y)  # log(1 + y) for RMSLE-friendly training

for idx_train, idx_valid in cv.split(X, duration_bins):
    X_train, y_train = X.iloc[idx_train].copy(), y_log.iloc[idx_train]
    X_valid, y_valid = X.iloc[idx_valid].copy(), y_log.iloc[idx_valid]
    X_test = test[X.columns].copy()

    # Target Encoding for categorical column
    encoder = TargetEncoder(random_state=42)
    X_train['Sex'] = encoder.fit_transform(X_train[['Sex']], y_train)[:, 0]
    X_valid['Sex'] = encoder.transform(X_valid[['Sex']])[:, 0]
    X_test['Sex'] = encoder.transform(X_test[['Sex']])[:, 0]

    model = xgb.XGBRegressor(
        n_estimators=2000,
        max_depth=10,
        learning_rate=0.03,
        subsample=0.7,
        colsample_bytree=0.7,
        objective='reg:squarederror',  # L2 loss
        eval_metric='rmse',
        early_stopping_rounds=100,
        random_state=42,
        verbosity=1,
    )

    model.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        verbose=100
    )

    y_pred += np.expm1(model.predict(X_test))

# Average predictions over folds
y_pred /= cv.get_n_splits()

# Create submission file
pred_xgb = y_pred
sample_submission = pd.read_csv('sample_submission.csv')
submission_xgb = pd.DataFrame({'id': sample_submission.id, 'Calories': pred_xgb})
submission_xgb.to_csv('submission_xgb_new.csv', index=False)

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import TargetEncoder
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import numpy as np
import pandas as pd

# Setup
drop_cols = ['id', 'Calories']
X = train.drop(columns=drop_cols)
y = train['Calories']
cat_features = ['Sex','Duration','Age']

# Stratification based on binned Duration
duration_bins = pd.qcut(X['Duration'], q=10, labels=False, duplicates='drop')

# Stratified K-Fold
cv = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)

# Log-transform the target
y_log = np.log1p(y)

# Initialize predictions
y_pred = np.zeros(len(test))
oof_preds = np.zeros(len(train))  # To store OOF predictions

for idx_fold, (idx_train, idx_valid) in enumerate(cv.split(X, duration_bins)):
    print(f"\nFold {idx_fold + 1}")
    
    X_train, y_train_fold = X.iloc[idx_train].copy(), y_log.iloc[idx_train]
    X_valid, y_valid_fold = X.iloc[idx_valid].copy(), y_log.iloc[idx_valid]
    X_test = test[X.columns].copy()

    # Target encoding
    encoder = TargetEncoder(random_state=42)
    X_train['Sex'] = encoder.fit_transform(X_train[['Sex']], y_train_fold)[:, 0]
    X_valid['Sex'] = encoder.transform(X_valid[['Sex']])[:, 0]
    X_test['Sex'] = encoder.transform(X_test[['Sex']])[:, 0]

    # Model
    model = xgb.XGBRegressor(
        n_estimators=2000,
        max_depth=10,
        learning_rate=0.03,
        subsample=0.7,
        colsample_bytree=0.7,
        objective='reg:squarederror',
        eval_metric='rmse',
        early_stopping_rounds=100,
        random_state=42,
        verbosity=1,
    )

    model.fit(
        X_train, y_train_fold,
        eval_set=[(X_valid, y_valid_fold)],
        verbose=100
    )

    # Predict on test and validation sets
    y_pred += np.expm1(model.predict(X_test))
    oof_preds[idx_valid] = np.expm1(model.predict(X_valid))

# Final test prediction (average)
y_pred /= cv.get_n_splits()

# OOF RMSLE
rmsle = np.sqrt(mean_squared_error(np.log1p(train['Calories']), np.log1p(oof_preds)))
print(f"\nOOF RMSLE: {rmsle:.5f}")

# Submission
sample_submission = pd.read_csv('sample_submission.csv')
submission_xgb = pd.DataFrame({'id': sample_submission.id, 'Calories': y_pred})
submission_xgb.to_csv('submission_xgb_new.csv', index=False)

In [ ]:
OOF RMSLE: 0.05991 => with ratio features
OOF RMSLE: 0.05993 => with ratio features + efficiency
OOF RMSLE: 0.05988 => with ratio features + efficiency + interation
OOF RMSLE: 0.05985 => with ratio features + efficiency + interation + pairwise polynomial
OOF RMSLE: 0.05994 => with ratio features +            + interation + pairwise polynomial


OOF RMSLE: 0.05983 => with ratio features +  effciency + interation + pairwise polynomial with Duration as a cxategory
OOF RMSLE: 0.05983 => with ratio features +  effciency + interation + pairwise polynomial with Duration and Age as a cxategory



In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import TargetEncoder
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import numpy as np
import pandas as pd

# Setup
drop_cols = ['id', 'Calories']
X = train.drop(columns=drop_cols)
y = train['Calories']
cat_features = ['Sex','Duration','Age']
categorical_cols = ['Sex','Duration']#,'Age']
# Stratification based on binned Duration
duration_bins = pd.qcut(X['Duration'], q=10, labels=False, duplicates='drop')

# Stratified K-Fold
cv = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)

# Log-transform the target
y_log = np.log1p(y)

# Initialize predictions
y_pred = np.zeros(len(test))
oof_preds = np.zeros(len(train))  # To store OOF predictions

for idx_fold, (idx_train, idx_valid) in enumerate(cv.split(X, duration_bins)):
    print(f"\nFold {idx_fold + 1}")
    
    X_train, y_train_fold = X.iloc[idx_train].copy(), y_log.iloc[idx_train]
    X_valid, y_valid_fold = X.iloc[idx_valid].copy(), y_log.iloc[idx_valid]
    X_test = test[X.columns].copy()

    encoder = TargetEncoder(random_state=42)
    X_train[categorical_cols] = encoder.fit_transform(X_train[categorical_cols], y_train_fold)
    X_valid[categorical_cols] = encoder.transform(X_valid[categorical_cols])
    X_test[categorical_cols] = encoder.transform(X_test[categorical_cols])

    # Model
    model = xgb.XGBRegressor(
        n_estimators=2000,
        max_depth=10,
        learning_rate=0.03,
        subsample=0.7,
        colsample_bytree=0.7,
        objective='reg:squarederror',
        eval_metric='rmse',
        early_stopping_rounds=100,
        random_state=42,
        verbosity=1,
    )

    model.fit(
        X_train, y_train_fold,
        eval_set=[(X_valid, y_valid_fold)],
        verbose=100
    )

    # Predict on test and validation sets
    y_pred += model.predict(X_test)
    oof_preds[idx_valid] = np.expm1(model.predict(X_valid))

# Final test prediction (average)
y_pred = np.expm1(y_pred / cv.get_n_splits()) 

# OOF RMSLE
rmsle = np.sqrt(mean_squared_error(np.log1p(train['Calories']), np.log1p(oof_preds)))
print(f"\nOOF RMSLE: {rmsle:.5f}")

# Submission
sample_submission = pd.read_csv('sample_submission.csv')
submission_xgb = pd.DataFrame({'id': sample_submission.id, 'Calories': y_pred})
submission_xgb.to_csv('submission_xgb_category.csv', index=False)

In [13]:
from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostRegressor, Pool
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

drop_cols = ['id', 'Calories']
X = train.drop(columns=drop_cols)
y = train['Calories']
# Setup
X_train = X
y_train = y
X_test = test.drop(columns=['id'])
categorical_cols = ['Sex']#,'Duration','Age']

# Stratify on Duration bins
duration_bins = pd.qcut(train['Duration'], q=10, labels=False)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Initialize prediction arrays
test_preds_log = np.zeros(len(test))       # accumulate raw predictions (log)
test_preds_lin = np.zeros(len(test))       # accumulate converted predictions
oof_preds = np.zeros(len(train))           # OOF predictions for validation

#X_train['Duration'] = X_train['Duration'].astype(int)
#X_test['Duration'] = X_test['Duration'].astype(int)
# Cross-validation loop
for fold, (idx_tr, idx_val) in enumerate(cv.split(X_train, duration_bins)):
    print(f'\nFold {fold + 1}')
    
    X_tr, X_val = X_train.iloc[idx_tr], X_train.iloc[idx_val]
    y_tr, y_val = np.log1p(y_train.iloc[idx_tr]), np.log1p(y_train.iloc[idx_val])
    
    train_pool = Pool(X_tr, y_tr, cat_features=categorical_cols)
    val_pool = Pool(X_val, y_val, cat_features=categorical_cols)
    test_pool = Pool(X_test, cat_features=categorical_cols)


    model = CatBoostRegressor(
        iterations=2000,
        learning_rate=0.03,
        depth=10,
        random_strength=5,
        l2_leaf_reg=7,
        bootstrap_type='Bernoulli',
        grow_policy='SymmetricTree',
        eval_metric='RMSE',
        random_seed=42,
        early_stopping_rounds=50,
        verbose=100
    )
    
    model.fit(train_pool, eval_set=val_pool)
    
    # Predict on test
    preds_test_log = model.predict(test_pool)
    preds_test_lin = np.expm1(preds_test_log)

    test_preds_log += preds_test_log
    test_preds_lin += preds_test_lin

    # OOF predictions for this fold
    oof_preds[idx_val] = np.expm1(model.predict(val_pool))

# Average predictions
test_preds_log = np.expm1(test_preds_log / cv.get_n_splits())  # average in log space
test_preds_lin = test_preds_lin / cv.get_n_splits()            # average in linear space

# Compute OOF RMSLE
rmsle_oof = np.sqrt(mean_squared_error(np.log1p(y_train), np.log1p(oof_preds)))
print(f"\nOOF RMSLE: {rmsle_oof:.5f}")

# Save predictions
test['Calories'] = test_preds_lin
test[['id', 'Calories']].to_csv('catboost_linear_avg_new_age.csv', index=False)

test['Calories'] = test_preds_log
test[['id', 'Calories']].to_csv('catboost_log_avg_new_age.csv', index=False)


Fold 1
0:	learn: 0.9365528	test: 0.9353485	best: 0.9353485 (0)	total: 117ms	remaining: 3m 52s
100:	learn: 0.0877443	test: 0.0879699	best: 0.0879699 (100)	total: 5.39s	remaining: 1m 41s
200:	learn: 0.0619111	test: 0.0629707	best: 0.0629707 (200)	total: 10.8s	remaining: 1m 36s
300:	learn: 0.0604706	test: 0.0619172	best: 0.0619172 (300)	total: 16.1s	remaining: 1m 30s
400:	learn: 0.0598075	test: 0.0615256	best: 0.0615256 (400)	total: 22.3s	remaining: 1m 29s
500:	learn: 0.0592577	test: 0.0612432	best: 0.0612432 (500)	total: 27.7s	remaining: 1m 22s
600:	learn: 0.0587654	test: 0.0610287	best: 0.0610287 (600)	total: 33.8s	remaining: 1m 18s
700:	learn: 0.0583421	test: 0.0608917	best: 0.0608917 (700)	total: 39.5s	remaining: 1m 13s
800:	learn: 0.0579884	test: 0.0607977	best: 0.0607977 (800)	total: 45.4s	remaining: 1m 7s
900:	learn: 0.0576507	test: 0.0607440	best: 0.0607440 (900)	total: 52.9s	remaining: 1m 4s
1000:	learn: 0.0573608	test: 0.0606797	best: 0.0606794 (999)	total: 58.3s	remaining: 58.

In [14]:
X

,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,bmi,dur_body_heart,BMR,...,Age_x_Duration,Age_x_Heart_Rate,Age_x_Weight,Age_x_Height,Duration_x_Heart_Rate,Duration_x_Weight,Duration_x_Height,Heart_Rate_x_Weight,Heart_Rate_x_Height,Weight_x_Height
0,male,36,189.0,82.0,26.0,101.0,41.0,22.955684,107666.0,1826.25,...,936.0,3636.0,2952.0,6804.0,2626.0,2132.0,4914.0,8282.0,19089.0,15498.0
1,female,64,163.0,60.0,8.0,85.0,39.7,22.582709,26996.0,1137.75,...,512.0,5440.0,3840.0,10432.0,680.0,480.0,1304.0,5100.0,13855.0,9780.0
2,female,51,161.0,64.0,7.0,84.0,39.8,24.690405,23402.4,1230.25,...,357.0,4284.0,3264.0,8211.0,588.0,448.0,1127.0,5376.0,13524.0,10304.0
3,male,20,192.0,90.0,25.0,105.0,40.7,24.414062,106837.5,2005.00,...,500.0,2100.0,1800.0,3840.0,2625.0,2250.0,4800.0,9450.0,20160.0,17280.0
4,female,38,166.0,61.0,25.0,102.0,40.6,22.136740,103530.0,1296.50,...,950.0,3876.0,2318.0,6308.0,2550.0,1525.0,4150.0,6222.0,16932.0,10126.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749995,male,28,193.0,97.0,30.0,114.0,40.9,26.040968,139878.0,2041.25,...,840.0,3192.0,2716.0,5404.0,3420.0,2910.0,5790.0,11058.0,22002.0,18721.0
749996,female,64,165.0,63.0,18.0,92.0,40.5,23.140496,67068.0,1180.25,...,1152.0,5888.0,4032.0,10560.0,1656.0,1134.0,2970.0,5796.0,15180.0,10395.0
749997,male,60,162.0,67.0,29.0,113.0,40.9,25.529645,134029.3,1387.50,...,1740.0,6780.0,4020.0,9720.0,3277.0,1943.0,4698.0,7571.0,18306.0,10854.0
749998,male,45,182.0,91.0,17.0,102.0,40.3,27.472527,69880.2,1827.50,...,765.0,4590.0,4095.0,8190.0,1734.0,1547.0,3094.0,9282.0,18564.0,16562.0


In [ ]:
# Save predictions
test['Calories'] = test_preds_lin
test[['id', 'Calories']].to_csv('catboost_linear_avg_age_cat.csv', index=False)

test['Calories'] = test_preds_log
test[['id', 'Calories']].to_csv('catboost_log_avg_age_cat.csv', index=False)

In [ ]:
X_test[categorical_cols].value_counts()

In [ ]:
test.head()

# Ensemble

In [16]:
data1=pd.read_csv('submission_xgb_new (1).csv')
data2=pd.read_csv('catboost_log_avg (2).csv')

data = data1.merge(data2, on='id', suffixes=('_xgb', '_cat'))


# 1. Simple average of the two predictions
data['Calories_avg'] = (data['Calories_xgb'] + data['Calories_cat']) / 2

# 2. Log-scale average (RMSLE-style averaging)
data['Calories_log_avg'] = np.expm1(
    (np.log1p(data['Calories_xgb']) + np.log1p(data['Calories_cat'])) / 2
)

print(data[['id', 'Calories_xgb', 'Calories_cat', 'Calories_avg', 'Calories_log_avg']].head())
data[['id', 'Calories_avg']].rename(columns={'Calories_avg': 'Calories'}).to_csv('submission_reg_avg_2.csv', index=False)

# Save log-scale average with 'Calories' as the second column
data[['id', 'Calories_log_avg']].rename(columns={'Calories_log_avg': 'Calories'}).to_csv('submission_log_avg_2.csv', index=False)

       id  Calories_xgb  Calories_cat  Calories_avg  Calories_log_avg
0  750000     27.541962     27.134063     27.338013         27.337279
1  750001    108.137621    107.451112    107.794366        107.793825
2  750002     87.279698     88.094149     87.686924         87.685989
3  750003    125.480219    125.700810    125.590514        125.590466
4  750004     76.096430     75.849336     75.972883         75.972784
